In [1]:
library(tidyr)
library(dplyr)
library(lubridate)

load = "Load_history.csv"
temp = "temperature_history.csv" 
solution = "Load_solution.csv"

setClass("num.with.commas")
setAs("character", "num.with.commas", function(from) as.numeric(gsub(",", "", from) ) )

loadRawClasses = c('factor', rep("numeric", 3), rep("num.with.commas", 24))

df = read.csv(load, stringsAsFactors=FALSE, colClasses=loadRawClasses)

Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union



In [2]:
head(df)

zone_id,year,month,day,h1,h2,h3,h4,h5,h6,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
1,2004,1,1,16853,16450,16517,16873,17064,17727,...,13518,13138,14130,16809,18150,18235,17925,16904,16162,14750
1,2004,1,2,14155,14038,14019,14489,14920,16072,...,16127,15448,15839,17727,18895,18650,18443,17580,16467,15258
1,2004,1,3,14439,14272,14109,14081,14775,15491,...,13507,13414,13826,15825,16996,16394,15406,14278,13315,12424
1,2004,1,4,11273,10415,9943,9859,9881,10248,...,14207,13614,14162,16237,17430,17218,16633,15238,13580,11727
1,2004,1,5,10750,10321,10107,10065,10419,12101,...,13845,14350,15501,17307,18786,19089,19192,18416,17006,16018
1,2004,1,6,15742,15682,16132,16761,17909,20234,...,18762,19162,21509,25314,28060,28768,28919,28653,27406,26507


In [25]:
tidyLoadDf = df %>%  gather(Hour, Consumption, h1:h24) %>%
                 mutate(Hour=as.numeric(substr(Hour, 2, nchar(Hour)))-1) %>%
                 unite(Date, year, month, day, sep='-') %>%
                 unite(DateTime, Date, Hour, sep=' ') %>%
                 mutate(DateTime=as.POSIXct(strptime(DateTime, "%Y-%m-%d %H", tz = "GMT"))) %>%
                 arrange(DateTime, zone_id)

In [26]:
#Spead consumption for each zone into seperate columns, so that we can easily select consumption for each zone later
tidyLoadDf = tidyLoadDf %>% mutate(zone_id=paste0('zone.',as.character(zone_id))) %>%
                      spread(zone_id, Consumption, fill = NA, convert = FALSE)
#Remove 2008/June data, because it's not complete, Create Zone 21 as sum of 20 zones consumption
tidyLoadDf = tidyLoadDf %>% filter(DateTime<as.POSIXct(strptime("2008-07-08 00:00:00", "%Y-%m-%d %H:%M:%S")))
#Add total consumption to zone.21
tidyLoadDf %>% select(zone.1:zone.20) %>% rowSums(na.rm=TRUE) -> tidyLoadDf$zone.21

In [27]:
tail(tidyLoadDf)

,DateTime,zone.1,zone.10,zone.11,zone.12,zone.13,zone.14,zone.15,zone.16,zone.17,...,zone.2,zone.20,zone.21,zone.3,zone.4,zone.5,zone.6,zone.7,zone.8,zone.9
1505,2008-07-07 16:00:00,26418,125860,196311,257577,20936,31836,84629,42909,45822,...,244890,112540,1602382,264237,488,11612,256503,264237,4049,20790
1506,2008-07-07 17:00:00,24067,125924,199347,264307,20491,33418,81089,44543,47149,...,233427,110340,1611099,251868,526,11679,245106,251868,4180,22113
1507,2008-07-07 18:00:00,24199,124208,196250,261670,23412,37700,89591,48807,49613,...,219468,115551,1645633,236807,548,10641,230109,236807,4228,63084
1508,2008-07-07 19:00:00,23831,119105,185775,246165,22815,36232,84512,46373,47985,...,210476,111560,1567266,227104,553,9960,220436,227104,4239,75369
1509,2008-07-07 20:00:00,23122,113794,174968,228722,22420,34607,79897,43641,47215,...,201663,108309,1479708,217594,539,9341,211004,217594,4287,75600
1510,2008-07-07 21:00:00,22305,110332,167040,220612,21852,32921,78252,41535,45423,...,196691,106030,1418537,212229,546,8880,205570,212229,4256,76356


In [28]:
ensembleClasses = c('NULL', 'factor', rep("numeric", 27))
df = read.csv(solution, stringsAsFactors=FALSE, colClasses=ensembleClasses)
tidyEnsembleDf = df %>%  gather(Hour, Consumption, h1:h24) %>%
                 mutate(Hour=as.numeric(substr(Hour, 2, nchar(Hour)))-1) %>%
                 unite(Date, year, month, day, sep='-') %>%
                 unite(DateTime, Date, Hour, sep=' ') %>%
                 mutate(DateTime=as.POSIXct(strptime(DateTime, "%Y-%m-%d %H", tz = "GMT"))) %>%
                 arrange(DateTime) %>%
                 mutate(zone_id=paste0('zone.',as.character(zone_id))) %>%
                 spread(zone_id, Consumption, fill = NA, convert = FALSE) %>% #Remove forecast, only need backcast
                 filter(DateTime<as.POSIXct(strptime("2008-07-08 00:00:00", "%Y-%m-%d %H:%M:%S")))

In [29]:
tail(tidyEnsembleDf)

,DateTime,zone.1,zone.10,zone.11,zone.12,zone.13,zone.14,zone.15,zone.16,zone.17,...,zone.2,zone.20,zone.21,zone.3,zone.4,zone.5,zone.6,zone.7,zone.8,zone.9
1505,2008-07-07 16:00:00,26418,125860,196311,257577,20936,31836,84629,42909,45822,...,244890,112540,2424298,264237,488,11612,256503,264237,4049,20790
1506,2008-07-07 17:00:00,24067,125924,199347,264307,20491,33418,81089,44543,47149,...,233427,110340,2398439,251868,526,11679,245106,251868,4180,22113
1507,2008-07-07 18:00:00,24199,124208,196250,261670,23412,37700,89591,48807,49613,...,219468,115551,2427857,236807,548,10641,230109,236807,4228,63084
1508,2008-07-07 19:00:00,23831,119105,185775,246165,22815,36232,84512,46373,47985,...,210476,111560,2332031,227104,553,9960,220436,227104,4239,75369
1509,2008-07-07 20:00:00,23122,113794,174968,228722,22420,34607,79897,43641,47215,...,201663,108309,2215667,217594,539,9341,211004,217594,4287,75600
1510,2008-07-07 21:00:00,22305,110332,167040,220612,21852,32921,78252,41535,45423,...,196691,106030,2138603,212229,546,8880,205570,212229,4256,76356


In [30]:
fullDf = tidyLoadDf
fullDf[is.na(fullDf$zone.1), ] = tidyEnsembleDf[, names(fullDf)]

In [31]:
tail(fullDf)

,DateTime,zone.1,zone.10,zone.11,zone.12,zone.13,zone.14,zone.15,zone.16,zone.17,...,zone.2,zone.20,zone.21,zone.3,zone.4,zone.5,zone.6,zone.7,zone.8,zone.9
1505,2008-07-07 16:00:00,26418,125860,196311,257577,20936,31836,84629,42909,45822,...,244890,112540,1602382,264237,488,11612,256503,264237,4049,20790
1506,2008-07-07 17:00:00,24067,125924,199347,264307,20491,33418,81089,44543,47149,...,233427,110340,1611099,251868,526,11679,245106,251868,4180,22113
1507,2008-07-07 18:00:00,24199,124208,196250,261670,23412,37700,89591,48807,49613,...,219468,115551,1645633,236807,548,10641,230109,236807,4228,63084
1508,2008-07-07 19:00:00,23831,119105,185775,246165,22815,36232,84512,46373,47985,...,210476,111560,1567266,227104,553,9960,220436,227104,4239,75369
1509,2008-07-07 20:00:00,23122,113794,174968,228722,22420,34607,79897,43641,47215,...,201663,108309,1479708,217594,539,9341,211004,217594,4287,75600
1510,2008-07-07 21:00:00,22305,110332,167040,220612,21852,32921,78252,41535,45423,...,196691,106030,1418537,212229,546,8880,205570,212229,4256,76356


In [32]:
write.csv(fullDf, "complete.csv", row.names=FALSE)